In [1]:
#pip install lightgbm
#conda install -c conda-forge lightgbm


In [2]:
import joblib
import pandas as pd
import threading
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from concurrent.futures import ThreadPoolExecutor

In [3]:
print(lgb.__version__)


4.5.0


In [4]:
data = pd.read_csv("DatasetInitalComplete.csv", low_memory=False)
data = data.drop(["7","10","11","13","14",'mel_mitotic_index', 'mel_thick_mm',
       'tbp_lv_dnn_lesion_confidence'],axis = 1)

In [5]:
data.shape

(400000, 50)

In [6]:
data

,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,...,1,2,3,4,5,6,8,9,12,15
0,1,80.0,0.0,0.0,9.27,18.093367,13.054772,19.090458,21.211776,26.302386,...,0.992879,1.614020,0.472059,5.650503,0.037528,0.952930,0.240838,37.515355,5.879538,0.141073
1,1,75.0,1.0,3.0,3.88,26.187010,21.394010,25.553420,24.914770,36.588750,...,0.998616,1.438920,0.526167,4.973141,0.088282,0.991787,0.315452,29.518401,4.764154,0.181296
2,1,80.0,1.0,0.0,6.55,33.480140,24.249430,26.160100,25.295430,42.488470,...,0.976854,2.008492,0.403345,8.597085,0.034536,0.849163,0.192565,56.186421,7.140923,0.150849
3,1,60.0,1.0,4.0,5.27,25.872050,20.194650,26.805590,28.459890,37.254570,...,0.973226,1.863920,0.404943,6.184975,0.042374,0.832057,0.194337,38.999721,6.448128,0.180971
4,1,55.0,1.0,1.0,5.29,30.966662,22.008635,24.066940,26.734379,39.219278,...,0.980691,1.882010,0.409715,6.650251,0.038820,0.877775,0.197552,42.312589,6.516821,0.165631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,0,60.0,0.0,1.0,3.10,20.396390,8.700477,19.498260,22.526430,28.216930,...,0.993499,0.700879,0.693530,1.181281,0.080221,0.964553,0.436643,6.484213,2.356538,0.211512
399996,0,65.0,1.0,4.0,2.66,18.325057,14.385684,27.391522,25.084622,32.956080,...,0.974921,1.771508,0.421346,5.659397,0.039210,0.857096,0.208423,32.171421,5.665769,0.164087
399997,0,65.0,0.0,4.0,4.66,21.155608,17.250798,27.887630,23.743460,35.003995,...,0.969403,1.930678,0.393691,6.472789,0.040591,0.806044,0.183824,41.111117,6.841718,0.177144
399998,0,50.0,1.0,4.0,2.62,20.209510,16.428130,34.063210,31.104090,39.607160,...,0.971324,1.683367,0.430196,4.941759,0.044474,0.823461,0.211837,30.236066,5.675103,0.182123


In [7]:
data.columns

Index(['target', 'age_approx', 'sex', 'anatom_site_general',
       'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B',
       'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext',
       'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', '0', '1', '2', '3', '4', '5', '6', '8', '9', '12', '15'],
      dtype='object')

In [8]:
non_numerical_columns = data.select_dtypes(exclude=['number']).columns

# Convert to a list if needed
non_numerical_columns_list = list(non_numerical_columns)

print(non_numerical_columns_list)

[]


In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [10]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [11]:
import pickle

data_new = data
data1 = {}
save_file = open("cases.dat", "wb")

for j in non_numerical_columns_list:
    a = list(set(list(data_new[j])))
    index = 0
    dic = {}
    for i in a:
        dic[i] = index
        index += 1

    data1[j] = dic

pickle.dump(data1, save_file)
save_file.close()

In [12]:
data_file = open("cases.dat", "rb")
cases = pickle.load(data_file)
type(cases)
data_file.close()

In [13]:
for i in cases:
    for j in cases[i]:
        data_new[i] = data_new[i].replace(j, cases[i][j])
        

In [14]:
classes = []
for column in non_numerical_columns_list:
    data[column] = le.fit_transform(data[column])
    print(le.classes_)
    classes.append(le.classes_)

In [15]:
import pickle
data_file = open("cases.dat", "rb")
cases = pickle.load(data_file)
type(cases)
data_file.close()

In [16]:
for i in cases:
    for j in cases[i]:
        data_new[i] = data_new[i].replace(j, cases[i][j])

In [17]:
data_new

,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,...,1,2,3,4,5,6,8,9,12,15
0,1,80.0,0.0,0.0,9.27,18.093367,13.054772,19.090458,21.211776,26.302386,...,0.992879,1.614020,0.472059,5.650503,0.037528,0.952930,0.240838,37.515355,5.879538,0.141073
1,1,75.0,1.0,3.0,3.88,26.187010,21.394010,25.553420,24.914770,36.588750,...,0.998616,1.438920,0.526167,4.973141,0.088282,0.991787,0.315452,29.518401,4.764154,0.181296
2,1,80.0,1.0,0.0,6.55,33.480140,24.249430,26.160100,25.295430,42.488470,...,0.976854,2.008492,0.403345,8.597085,0.034536,0.849163,0.192565,56.186421,7.140923,0.150849
3,1,60.0,1.0,4.0,5.27,25.872050,20.194650,26.805590,28.459890,37.254570,...,0.973226,1.863920,0.404943,6.184975,0.042374,0.832057,0.194337,38.999721,6.448128,0.180971
4,1,55.0,1.0,1.0,5.29,30.966662,22.008635,24.066940,26.734379,39.219278,...,0.980691,1.882010,0.409715,6.650251,0.038820,0.877775,0.197552,42.312589,6.516821,0.165631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,0,60.0,0.0,1.0,3.10,20.396390,8.700477,19.498260,22.526430,28.216930,...,0.993499,0.700879,0.693530,1.181281,0.080221,0.964553,0.436643,6.484213,2.356538,0.211512
399996,0,65.0,1.0,4.0,2.66,18.325057,14.385684,27.391522,25.084622,32.956080,...,0.974921,1.771508,0.421346,5.659397,0.039210,0.857096,0.208423,32.171421,5.665769,0.164087
399997,0,65.0,0.0,4.0,4.66,21.155608,17.250798,27.887630,23.743460,35.003995,...,0.969403,1.930678,0.393691,6.472789,0.040591,0.806044,0.183824,41.111117,6.841718,0.177144
399998,0,50.0,1.0,4.0,2.62,20.209510,16.428130,34.063210,31.104090,39.607160,...,0.971324,1.683367,0.430196,4.941759,0.044474,0.823461,0.211837,30.236066,5.675103,0.182123


In [18]:
df_target_1 = data[data['target'] == 1]
df_target_0 = data[data['target'] == 0]

# Select the last 40 positive and negative samples for the test set
test_positive = df_target_1.tail(40)
test_negative = df_target_0.tail(40)

# Remove the last 40 samples from df_target_1 and df_target_0 (creating new DataFrames)
df_target_1 = df_target_1.iloc[:-40]  # All rows except the last 40 positives
df_target_0 = df_target_0.iloc[:-40]  # All rows except the last 40 negatives

# Concatenate the last 40 positives and negatives into the test set
test_set = pd.concat([test_positive, test_negative], ignore_index=True)

# Remaining samples in df_target_1 and df_target_0 are now updated
test_set

,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,...,1,2,3,4,5,6,8,9,12,15
0,1,55.0,1.0,0.0,9.53,23.854059,15.407355,20.493286,22.711772,31.448226,...,0.982981,1.928518,0.412657,7.512085,0.036913,0.893531,0.199944,47.151497,6.746231,0.156583
1,1,80.0,1.0,2.0,16.40,16.371787,15.457629,16.975724,20.511455,23.584118,...,0.995487,1.553769,0.464449,4.505377,0.036223,0.972531,0.229453,27.574264,5.476051,0.142125
2,1,55.0,0.0,0.0,7.48,25.711526,17.473305,25.697414,25.440119,36.351611,...,0.987482,1.703090,0.438763,5.360729,0.039874,0.924270,0.220147,32.541218,5.735000,0.160379
3,1,65.0,1.0,3.0,1.07,28.581466,25.153961,33.832074,31.270528,44.288931,...,0.974787,1.699724,0.426182,5.014497,0.041026,0.835442,0.200962,32.700228,6.173051,0.174352
4,1,65.0,0.0,4.0,1.10,26.453680,23.072678,19.165143,20.453265,32.666495,...,0.936247,2.815704,0.318144,15.612487,0.032847,0.610805,0.147875,95.447970,9.121615,0.164574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,60.0,0.0,1.0,3.10,20.396390,8.700477,19.498260,22.526430,28.216930,...,0.993499,0.700879,0.693530,1.181281,0.080221,0.964553,0.436643,6.484213,2.356538,0.211512
76,0,65.0,1.0,4.0,2.66,18.325057,14.385684,27.391522,25.084622,32.956080,...,0.974921,1.771508,0.421346,5.659397,0.039210,0.857096,0.208423,32.171421,5.665769,0.164087
77,0,65.0,0.0,4.0,4.66,21.155608,17.250798,27.887630,23.743460,35.003995,...,0.969403,1.930678,0.393691,6.472789,0.040591,0.806044,0.183824,41.111117,6.841718,0.177144
78,0,50.0,1.0,4.0,2.62,20.209510,16.428130,34.063210,31.104090,39.607160,...,0.971324,1.683367,0.430196,4.941759,0.044474,0.823461,0.211837,30.236066,5.675103,0.182123


In [19]:
#pip install matplotlib

In [21]:
##train_set, test_set = train_test_split(batch1, test_size=0.1, stratify=batch1['target'], random_state=42)
#scaler = StandardScaler()
#X_train = train_set.drop(columns=['target'])
#y_train = train_set['target']
#X_test = test_set.drop(columns=['target'])
#y_test = test_set['target']
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [23]:
tprsacc = []
final_predict_data = data.drop(["target"],axis=1)

def trainer(number,batch):
    #print(pd.DataFrame(batch))
    #print(number)
    scaler = StandardScaler()

    trainn_set, testn_set = train_test_split(batch, test_size=0.2, stratify=batch['target'], random_state=42)
    X_trainn = trainn_set.drop(columns=['target'])
    y_trainn = trainn_set['target']
    X_testn= testn_set.drop(columns=['target'])
    y_testn = testn_set['target']
    X_trainn = scaler.fit_transform(X_trainn)
    X_testn = scaler.transform(X_testn)
    
    model = lgb.LGBMClassifier(verbose=-1,class_type = 'balanced')
    model.fit(X_trainn, y_trainn)
    print("model"+str(number))
    name = "model" + str(number) + ".joblib"
    
    predictions = model.predict(X_testn)
    tn, fp, fn, tp = confusion_matrix(y_testn, predictions).ravel()
    accuracy = accuracy_score(y_testn,predictions)
    tprsacc.append((tp/(tp+fn),accuracy))

    joblib.dump(model, name)
    
    #predictions = model.predict(X_testn)
    #tn, fp, fn, tp = confusion_matrix(y_testn, predictions).ravel()
    
    #accuracy = accuracy_score(y_testn,predictions)
    #print("model number"+str(i)+" true positives: "+str(tp/(tp+fn)))
    #print("model number"+str(i)+" accuracu: "+str(accuracy))
    
    
    # Make predictions and evaluate
    


In [24]:
threads = []
import time

start_time = time.time()


for i in range(0,1018):
    batch = batch1 = pd.concat([df_target_1, df_target_0[i*352:(i*352)+352]], ignore_index=True)
       
    thread = threading.Thread(target=trainer, args=(i,batch))
    threads.append(thread)
    thread.start()
end_time = time.time()
total_time = end_time - start_time
total_time
#batch

model3
model0
model21
model12
model13
model7
model6
model5
model10
model8
model23
model4
model1
model11
model20
model19
model9
model18
model14
model22
model27
model25
model15
model16
model17
model26
model24
model31
model29
model28
model2
model37
model35
model30
model34
model36
model32
model40
model33
model38
model41
model43
model42
model48
model39
model44
model49
model46
model45
model50
model58
model47
model53
model52
model56
model60
model64
model61
model55
model59
model54
model57
model68
model65
model63
model75
model62
model74
model51
model67
model77
model79
model69
model66
model83
model72
model81
model86
model82
model89
model73
model71


34.774590492248535

model92
model85
model91
model102
model70
model106
model95
model97
model87
model98
model88
model76
model80
model101
model99
model112
model94
model78
model93
model103
model100
model90
model108
model107
model104
model109
model96
model111
model84
model131
model105
model119
model122
model113
model120
model130
model129
model125
model136
model117
model110
model126
model115
model127
model121
model116
model138
model150
model143
model114
model153
model132
model133
model147
model158
model123
model134
model139
model151
model148
model118
model128
model124
model176
model195
model161
model169
model141
model149
model142
model144
model157
model165
model135
model173
model155
model152
model178
model145
model201
model180
model211
model160
model181
model174
model182
model163
model154
model164
model194
model166
model168
model171
model217
model183
model184
model188
model146
model177
model196
model137
model215
model185model216

model140
model172
model200
model189
model156
model199
model207
model242
model159
m

In [25]:
tprsacc 

[(0.8285714285714286, 0.8581560283687943),
 (0.8714285714285714, 0.8439716312056738),
 (0.8571428571428571, 0.8439716312056738),
 (0.8285714285714286, 0.8297872340425532),
 (0.8285714285714286, 0.851063829787234),
 (0.8714285714285714, 0.8865248226950354),
 (0.8714285714285714, 0.851063829787234),
 (0.8571428571428571, 0.8581560283687943),
 (0.8428571428571429, 0.8297872340425532),
 (0.8857142857142857, 0.8723404255319149),
 (0.8428571428571429, 0.8723404255319149),
 (0.8714285714285714, 0.8652482269503546),
 (0.8428571428571429, 0.8794326241134752),
 (0.8285714285714286, 0.8368794326241135),
 (0.8142857142857143, 0.8581560283687943),
 (0.8285714285714286, 0.8297872340425532),
 (0.8571428571428571, 0.8652482269503546),
 (0.8428571428571429, 0.8581560283687943),
 (0.8285714285714286, 0.8439716312056738),
 (0.8714285714285714, 0.8936170212765957),
 (0.8571428571428571, 0.8439716312056738),
 (0.8285714285714286, 0.8581560283687943),
 (0.8428571428571429, 0.8652482269503546),
 (0.828571428

In [26]:
sorted_based_on_tp = max(tprsacc , key=lambda x: x[0])
sorted_based_on_tp

(0.9285714285714286, 0.8936170212765957)

In [27]:
sorted_based_on_acc = max(tprsacc , key=lambda x: x[1])
sorted_based_on_acc

(0.8714285714285714, 0.9219858156028369)

In [28]:
testing_predictions = {}

In [29]:
testing_predictions = {}
#testing_predictions_lock = threading.Lock()

def tester(data, model_name):
    scaler = StandardScaler()
    data = pd.DataFrame(scaler.fit_transform(data))
    for i in range(len(data)):
        # Prepare the test data
        test_datan = data.iloc[[i]]
        
        # Load the model
        #model_name = f"model{model_number}.joblib"
        model = joblib.load(model_name)
        
        # Predict using the model
        prediction = model.predict(test_datan)
        
        # Initialize the list for key `i` if it does not exist
        if i not in testing_predictions:
            testing_predictions[i] = []
        
        # Append the prediction to the list
        testing_predictions[i].append(prediction[0])


In [30]:
#x1test = x1testset.drop(columns=['target'])
#x0test = x0testset.drop(columns=['target'])
#y1testset = x1testset['target']
#y0testset = x0testset['target']
#xInput = pd.concat([x1test,x0test],ignore_index=True).head(400)
#yOutput = pd.concat([y1testset,y0testset],ignore_index=True).head(400)
test_set_targets = test_set['target']
test_set_input = test_set.drop(columns = ['target'])
nummod = 10
#models = [tprsacc[value] for value in tprsacc]
#print(models)
#print(tprsacc)
for i  in range(0,1000):
    model_name = name = "model" + str(i) + ".joblib"
    tester(test_set_input, model_name)

In [31]:
hhh = testing_predictions

In [32]:
hhh

{0: [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [34]:
for key, value in testing_predictions.items():
    # Check if the value is a list of lists, a list, or a single value
    if isinstance(value, list):
        # If it's a list of lists, flatten it
        if all(isinstance(sublist, list) for sublist in value):
            flat_predictions = [item for sublist in value for item in sublist]
        else:
            # If it's a simple list, use it directly
            flat_predictions = value
    else:
        # If it's a single value, convert it to a list
        flat_predictions = [value]
    
    # Convert to pandas Series
    prediction_series = pd.Series(flat_predictions)
    
    # Find the mode (most frequent value)
    mode_value = prediction_series.mode().iloc[0]
    
    # Replace the list with the mode value
    testing_predictions[key] = mode_value

print(testing_predictions)

{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 0, 20: 0, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 0, 27: 0, 28: 1, 29: 1, 30: 0, 31: 0, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 0, 38: 1, 39: 1, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 1, 49: 0, 50: 1, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 1, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 1, 69: 0, 70: 0, 71: 0, 72: 0, 73: 1, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0}


In [35]:
from itertools import chain
testing_predictions_list = list(chain.from_iterable(
    [value] if isinstance(value, (int, float, np.integer, np.floating)) else value 
    for value in testing_predictions.values()
))

print(testing_predictions_list)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


In [36]:
testing_predictions_list

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0]

In [37]:
print(testing_predictions)

{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 0, 20: 0, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 0, 27: 0, 28: 1, 29: 1, 30: 0, 31: 0, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 0, 38: 1, 39: 1, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 1, 49: 0, 50: 1, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 1, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 1, 69: 0, 70: 0, 71: 0, 72: 0, 73: 1, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0}


In [46]:
#X_test = pd.DataFrame(test_set_input)

# Select the "target" column from data_new using the indices of X_test
#@@y_test = data_new.loc[test_set_input, "target"]

In [40]:
tn, fp, fn, tp = confusion_matrix(test_set_targets, testing_predictions_list).ravel()
recall = tp/(tp+fn)
print("true positive rate:",tp/(tp+fn))
print("precision:",tp/(tp+fp))
precision = tp/(tp+fp)
accuracy = accuracy_score(test_set_targets,testing_predictions_list)
print(tp, tn , fp , fn)
print("accuracy:",accuracy)
fscore = 2/((1/recall)+(1/precision))
print("f score:",fscore)

true positive rate: 0.825
precision: 0.868421052631579
33 35 5 7
accuracy: 0.85
f score: 0.8461538461538463


In [41]:
from skimage.feature import graycomatrix, graycoprops

def feature_extractor(img,label):
    df = {}

    GSLM = graycomatrix(img, [1], [0])       
    GSLM_Energy = graycoprops(GSLM, 'energy')[0]
    df['0'] = GSLM_Energy[0]
    GSLM_corr = graycoprops(GSLM, 'correlation')[0]
    df['1'] = GSLM_corr[0]       
    GSLM_diss = graycoprops(GSLM, 'dissimilarity')[0]
    df['2'] = GSLM_diss[0]       
    GSLM_hom = graycoprops(GSLM, 'homogeneity')[0]
    df['3'] = GSLM_hom[0]       
    GSLM_contr = graycoprops(GSLM, 'contrast')[0]
    df['4'] = GSLM_contr[0]

    GSLM2 = graycomatrix(img, [3], [0])       
    GSLM_Energy2 = graycoprops(GSLM2, 'energy')[0]
    df['5'] = GSLM_Energy2[0]
    GSLM_corr2 = graycoprops(GSLM2, 'correlation')[0]
    df['6'] = GSLM_corr2[0]       
    GSLM_diss2 = graycoprops(GSLM2, 'dissimilarity')[0]
    df['7'] = GSLM_diss2[0]       
    GSLM_hom2 = graycoprops(GSLM2, 'homogeneity')[0]
    df['8'] = GSLM_hom2[0]       
    GSLM_contr2 = graycoprops(GSLM2, 'contrast')[0]
    df['9'] = GSLM_contr2[0]

    GSLM3 = graycomatrix(img, [5], [0])       
    GSLM_Energy3 = graycoprops(GSLM3, 'energy')[0]
    df['10'] = GSLM_Energy3[0]
    GSLM_corr3 = graycoprops(GSLM3, 'correlation')[0]
    df['11'] = GSLM_corr3[0]       
    GSLM_diss3 = graycoprops(GSLM3, 'dissimilarity')[0]
    df['12'] = GSLM_diss3[0]       
    GSLM_hom3 = graycoprops(GSLM3, 'homogeneity')[0]
    df['13'] = GSLM_hom3[0]       
    GSLM_contr3 = graycoprops(GSLM3, 'contrast')[0]
    df['14'] = GSLM_contr3[0]

    GSLM4 = graycomatrix(img, [0], [np.pi/4])       
    GSLM_Energy4 = graycoprops(GSLM4, 'energy')[0]
    df['15'] = GSLM_Energy4[0]
    
    return df

In [42]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io



In [43]:
kaggle_test_data = pd.read_csv("test-metadata.csv", low_memory=False)
#print(kaggle_test_data.columns)

features = pd.DataFrame(columns=['isic_id', 'patient_id', 'age_approx', 'sex', 'anatom_site_general',
       'clin_size_long_diam_mm', 'image_type', 'tbp_tile_type', 'tbp_lv_A',
       'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext',
       'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2',
       'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 'tbp_lv_deltaA',
       'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
       'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_location',
       'tbp_lv_location_simple', 'tbp_lv_minorAxisMM',
       'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color',
       'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL',
       'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle',
       'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z', 'attribution', 'copyright_license','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15'])
#print(len(features.columns))

# Create an empty DataFrame to store features
#features = pd.DataFrame()

# Open the HDF5 file
with h5py.File('test-image.hdf5', 'r') as hdf:
    index = 0

    # Iterate over each row in the CSV and extract corresponding image features
    for row in kaggle_test_data.values:
        # Extract the image from HDF5 (you may need to map row[0] or row[1] to image key in HDF5)
        img_key = row[0]  # Assuming the first column holds the image key; adjust accordingly.
        binary_data = hdf[img_key][()]  # Extract binary data for the image
        
        # Convert binary data to an image using PIL
        image = Image.open(io.BytesIO(binary_data))
        image_np = np.asarray(image)

        # Make sure the image is grayscale (for graycomatrix)
        if len(image_np.shape) == 3:
            image_np = image_np[..., 0]  # Convert to grayscale by taking the first channel

        # Extract features
        temp = list(row)
        temp.extend(feature_extractor(image_np, row[1]).values())
        #print(temp)
        features.loc[index] = temp
        index += 1
        #feature_values = list(feature_extractor(image_np, row[1]).values())  # row[1] might be label, adjust if needed
        
        # Append to the DataFrame
        #features.loc[index] = feature_values
        #index += 1

# Concatenate the features DataFrame with the kaggle_test_data
final_test_data = pd.concat([kaggle_test_data, features], axis=1)
final_test_data

,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,...,6,7,8,9,10,11,12,13,14,15
0,ISIC_0015657,IP_6074337,45.0,male,posterior torso,2.70,TBP tile: close-up,3D: XP,22.80433,20.007270,...,0.553873,7.459348,0.132819,96.550519,0.032018,0.365716,8.912912,0.113601,137.299437,0.171095
1,ISIC_0015729,IP_1664139,35.0,female,lower extremity,2.52,TBP tile: close-up,3D: XP,16.64867,9.657964,...,0.781837,3.604525,0.241643,21.532525,0.050054,0.698209,4.198800,0.217082,29.756533,0.201057
2,ISIC_0015740,IP_7142616,65.0,male,posterior torso,3.16,TBP tile: close-up,3D: XP,24.25384,19.937380,...,0.479872,11.642785,0.085032,225.317662,0.021538,0.299227,13.668362,0.072228,304.609244,0.138440


In [44]:
#print(data.columns)

final_test_data_filtered = final_test_data[['age_approx', 'sex', 'anatom_site_general',
       'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B',
       'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext',
       'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
       'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
       'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
       'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
       'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
       'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
       'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
       'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
       'tbp_lv_z', '0', '1', '2', '3', '4', '5', '6', '8',
       '9', '12', '15']]
final_test_data_filtered = final_test_data_filtered.loc[:, ~final_test_data_filtered.columns.duplicated()]

# Now final_test_data will have only unique columns
#data = data.drop(["7","10","11","13","14"],axis = 1)
#final_test_data
#final_test_data = final_test_data.drop(["7","10","11","13","14"],axis = 1)
final_test_data_filtered
# Now final_test_data_filtered contains only the columns in new_data
#final_test_data_filtered.head()

,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,...,1,2,3,4,5,6,8,9,12,15
0,45.0,male,posterior torso,2.70,22.80433,20.007270,28.38412,27.043640,36.410100,33.640000,...,0.894515,3.532928,0.259952,22.786626,0.033921,0.553873,0.132819,96.550519,8.912912,0.171095
1,35.0,female,lower extremity,2.52,16.64867,9.657964,31.31752,27.524318,35.467806,29.169579,...,0.938757,1.778194,0.433250,6.065032,0.053250,0.781837,0.241643,21.532525,4.198800,0.201057
2,65.0,male,posterior torso,3.16,24.25384,19.937380,30.46368,28.384240,38.939500,34.686660,...,0.866645,5.716565,0.171134,57.551773,0.022449,0.479872,0.085032,225.317662,13.668362,0.138440
